In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
import experiment_manager as em
import far_ho as far

Data folder is C:\Users\lfranceschi\DATASETS
Experiment save directory is  C:\Users\lfranceschi\EXPERIMENTS


In [3]:
far.utils.check()

3


In [23]:
try: ss.close()
except: pass
tf.reset_default_graph()
ss = tf.InteractiveSession()
v1 = tf.Variable([1.,3])

v2 = tf.Variable([[-1., -2], [1., -1.]])

In [24]:
v01 = far.get_hyperparameter('v10', [10., 21.])

In [25]:
lmbd = far.get_hyperparameter('lambda', 
                              initializer=tf.ones_initializer,
                             shape=v2.get_shape())

cost = tf.reduce_mean(v1**2) + tf.reduce_sum(lmbd*v2**2)

io_optim = far.MomentumOptimizer(far.get_hyperparameter('eta', 0.1), 
                                   far.get_hyperparameter('mu', .9))

io_optim_dict = io_optim.minimize(cost) 



In [26]:
io_optim_dict.dynamics

[(<tf.Variable 'Variable:0' shape=(2,) dtype=float32_ref>,
  <tf.Tensor 'sub:0' shape=(2,) dtype=float32>),
 (<tf.Variable 'Variable/Momentum:0' shape=(2,) dtype=float32_ref>,
  <tf.Tensor 'add_1:0' shape=(2,) dtype=float32>),
 (<tf.Variable 'Variable_1:0' shape=(2, 2) dtype=float32_ref>,
  <tf.Tensor 'sub_1:0' shape=(2, 2) dtype=float32>),
 (<tf.Variable 'Variable_1/Momentum:0' shape=(2, 2) dtype=float32_ref>,
  <tf.Tensor 'add_2:0' shape=(2, 2) dtype=float32>)]

In [27]:
io_optim_dict.set_init_dynamics({v1: v01})  # v1 is initialized with v01

In [28]:
oo = tf.reduce_mean(v1**2) + tf.reduce_mean(v2**2)
rhg = far.ReverseHg()
rhg.compute_gradients(oo, io_optim_dict)

[<tf.Variable 'v10:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'lambda:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'eta:0' shape=() dtype=float32_ref>,
 <tf.Variable 'mu:0' shape=() dtype=float32_ref>]

In [29]:
optim_oo = tf.train.AdamOptimizer(.01)
ts_hy = optim_oo.apply_gradients(rhg.hgrads_hvars())

In [30]:
tf.global_variables()

[<tf.Variable 'Variable:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'v10:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'lambda:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'eta:0' shape=() dtype=float32_ref>,
 <tf.Variable 'mu:0' shape=() dtype=float32_ref>,
 <tf.Variable 'Variable/Momentum:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'Variable_1/Momentum:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'beta1_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'beta2_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'v10/Adam:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'v10/Adam_1:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'lambda/Adam:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'lambda/Adam_1:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'eta/Adam:0' shape=() dtype=float32_ref>,
 <tf.Variable 'eta/Adam_1:0' shape=() dtype=float32_ref>,
 <tf.Variable 'mu/Adam:0' shape=() dtype=float32_ref

In [31]:
far.utils.hyperparameters()

[<tf.Variable 'v10:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'lambda:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'eta:0' shape=() dtype=float32_ref>,
 <tf.Variable 'mu:0' shape=() dtype=float32_ref>]

In [32]:
tf.global_variables_initializer().run()
ss.run(far.utils.hyperparameters())

[array([ 10.,  21.], dtype=float32), array([[ 1.,  1.],
        [ 1.,  1.]], dtype=float32), 0.1, 0.89999998]

In [33]:
ss.run(tf.global_variables())

[array([ 1.,  3.], dtype=float32), array([[-1., -2.],
        [ 1., -1.]], dtype=float32), array([ 10.,  21.], dtype=float32), array([[ 1.,  1.],
        [ 1.,  1.]], dtype=float32), 0.1, 0.89999998, array([ 0.,  0.], dtype=float32), array([[ 0.,  0.],
        [ 0.,  0.]], dtype=float32), 0.89999998, 0.99900001, array([ 0.,  0.], dtype=float32), array([ 0.,  0.], dtype=float32), array([[ 0.,  0.],
        [ 0.,  0.]], dtype=float32), array([[ 0.,  0.],
        [ 0.,  0.]], dtype=float32), 0.0, 0.0, 0.0, 0.0]

In [34]:
def print_history():
    for h in rhg._history:
        print(h)
        print('='*20)

In [35]:
rhg.run(5)

In [36]:
print_history()

[array([ 10.,  21.], dtype=float32), array([ 0.,  0.], dtype=float32), array([[-1., -2.],
       [ 1., -1.]], dtype=float32), array([[ 0.,  0.],
       [ 0.,  0.]], dtype=float32)]
[array([  9.        ,  18.89999962], dtype=float32), array([ 10.,  21.], dtype=float32), array([[-0.80000001, -1.60000002],
       [ 0.80000001, -0.80000001]], dtype=float32), array([[-2., -4.],
       [ 2., -2.]], dtype=float32)]
[array([  7.19999981,  15.11999989], dtype=float32), array([ 18.        ,  37.79999924], dtype=float32), array([[-0.46000001, -0.92000002],
       [ 0.46000001, -0.46000001]], dtype=float32), array([[-3.4000001 , -6.80000019],
       [ 3.4000001 , -3.4000001 ]], dtype=float32)]
[array([  4.85999966,  10.20600033], dtype=float32), array([ 23.39999771,  49.13999557], dtype=float32), array([[-0.06200001, -0.12400001],
       [ 0.06200001, -0.06200001]], dtype=float32), array([[-3.98000002, -7.96000004],
       [ 3.98000002, -3.98000002]], dtype=float32)]
[array([ 2.26799989,  4.762800

In [21]:
rhg.run(5)

In [22]:
print_history()

[array([ 10.,  21.], dtype=float32), array([ 0.,  0.], dtype=float32), array([[-1., -2.],
       [ 1.,  0.]], dtype=float32), array([[ 0.,  0.],
       [ 0.,  0.]], dtype=float32)]
[array([  9.        ,  18.89999962], dtype=float32), array([ 10.,  21.], dtype=float32), array([[-0.80000001, -1.60000002],
       [ 0.80000001,  0.        ]], dtype=float32), array([[-2., -4.],
       [ 2.,  0.]], dtype=float32)]
[array([  7.19999981,  15.11999989], dtype=float32), array([ 18.        ,  37.79999924], dtype=float32), array([[-0.46000001, -0.92000002],
       [ 0.46000001,  0.        ]], dtype=float32), array([[-3.4000001 , -6.80000019],
       [ 3.4000001 ,  0.        ]], dtype=float32)]
[array([  4.85999966,  10.20600033], dtype=float32), array([ 23.39999771,  49.13999557], dtype=float32), array([[-0.06200001, -0.12400001],
       [ 0.06200001,  0.        ]], dtype=float32), array([[-3.98000002, -7.96000004],
       [ 3.98000002,  0.        ]], dtype=float32)]
[array([ 2.26799989,  4.762800

In [ ]:
# THERE MUST BE SOME SHITTY PROBLEM WITH STATE........... THAT SOMEHOW GET CHANEGD
# SOMETHING GOES BAAD AT THE SECOND ITERATION.... FUCKING WHY?


# THERE IS SOME MAJOR PROPLEM WITH NEW IMPLEMENTATION...
NO IDEA WHAT'S GOING ON... SHITE

In [21]:
ss.run(far.utils.hypergradients())

[array([ 0.00850303,  0.01785634], dtype=float32),
 array([[ 0.20626384,  0.82505536],
        [ 0.20626384,  0.        ]], dtype=float32),
 133.51303,
 25.114243]

In [23]:
ss.run(far.utils.lagrangian_multipliers())

[array([ 0.00850303,  0.01785634], dtype=float32),
 array([ 0.06717408,  0.14106545], dtype=float32),
 array([[-0.16844368, -0.33688736],
        [ 0.16844368,  0.        ]], dtype=float32),
 array([[-0.03549819, -0.07099639],
        [ 0.03549819,  0.        ]], dtype=float32)]

In [307]:
# ss.run(ts_hy)  # perform an hypergradient descent step....
# print(ss.run(far.utils.hyperparameters()))
# print(ss.run(oo))

In [24]:
tf.get_collection(far.GraphKeys.HYPERGRADIENTS)

[<tf.Tensor 'add_3_1/gradients_3/add_3_1/Vectorization_2/Reshape_grad/Reshape:0' shape=(2,) dtype=float32>,
 <tf.Variable 'add_3/lambda/hypergradient:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'add_3/eta/hypergradient:0' shape=() dtype=float32_ref>,
 <tf.Variable 'add_3/mu/hypergradient:0' shape=() dtype=float32_ref>]

# check with rfho

In [308]:
import rfho as rf

In [309]:
w, c, co = rf.vectorize_model([v1, v2], cost, oo, augment=1)

In [310]:
dyn = rf.MomentumOptimizer.create(w, 0.1, 0.9, loss=c)

In [311]:
hyperg = rf.HyperOptimizer(dyn, {co: lmbd}, rf.ReverseHG)

In [312]:
hyperg.initialize()

True

In [313]:
hgs = hyperg.run(10, val_feed_dict_suppliers={co:lambda: None},
                 _debug_no_hyper_update=True)

In [314]:
ss.run(hyperg.hyper_gradients.hyper_gradient_vars)

[array([[ 0.2233436 ,  1.34006178],
        [-0.2233436 ,  0.        ]], dtype=float32)]

# END... RESULTS ARE (almost the) SAME (CFR cell 306)

In [209]:
rfho_hs = hyperg.hyper_gradients.w_hist

In [210]:
len(rfho_hs)

10

In [218]:
rfho_hs[9], rhg._history[10]  # in far version also last value is saved

(array([-24.56974983, -22.56974983, -21.56974983,   0.27419662,
          0.54839325,  -0.27419662,   0.        ,  44.92248917,
         44.92248917,  44.92248917,   2.48619032,   4.97238111,
         -2.48619032,   0.        ], dtype=float32),
 [array([-29.34610748, -27.34610748, -26.34610748], dtype=float32),
  array([ 47.76357651,  47.76357651,  47.76357651], dtype=float32),
  array([[-0.00439989, -0.00879972],
         [ 0.00439989,  0.        ]], dtype=float32),
  array([[ 2.78596425,  5.57192898],
         [-2.78596425,  0.        ]], dtype=float32)])

In [219]:
[1,2,4][:-1]

[1, 2]

In [222]:
ss.run(list(hyperg.hyper_gradients.p_dict.values()))

[array([  7.33333349e+00,   7.33333349e+00,   7.33333349e+00,
         -3.87176406e-05,  -7.74352811e-05,   3.87176406e-05,
          0.00000000e+00,  -4.29872227e+00,  -4.29872227e+00,
         -4.29872227e+00,  -1.10320083e-03,  -2.20641680e-03,
          1.10320083e-03,   0.00000000e+00], dtype=float32)]